Skrypt zawsze zaczynamy od importu bibliotek. Te, które przydadzą się dzisiaj, są importowane poniżej.
Użyte aliasy (`np`, `pd`, `plt`) są ogólnie przyjęte i powinno się używać takich, chociaż zdefiniowanie innych jest możliwe i w zasadzie nie jest błędem, jednak zmniejsza czytelność kodu.

In [1]:
#tak można zainstalować dodatkowe biblioteki w notebooku:
import sys
!{sys.executable} -m pip install tables

import numpy as np #podstawowe operacje numeryczne i matematyczne
import os #interakcje z systemem operacyjnym
import pandas as pd #analiza danych
import pickle #zapisywanie i wczytywanie z pliku
from pathlib import Path #łatwe manipulowanie ścieżkami
from glob import glob

## PODSTAWY PYTHONA - POWTÓRKA

### 1. *List comprehension* i *dictionary comprehension*

Na początek przypomnimy sobie *list comprehension*, czyli pythonowy sposób tworzenia listy elementów, które są zawarte w już istniejącej liście/secie/sekwencji/generatorze i spełniają jakieś zadane kryterium. Normalnie tworzylibyśmy taką nową listę przy użyciu pętli for. *List comprehension* ma w stosunku do standardowego, "pętlowego", zapisu kilka zalet - przede wszystkim jest bardziej czytelny i krótszy, więc zwiększa szybkość pisania kodu. (Oczywiście nie zawsze - w przypadku pętli zagnieżdżonych, zwłaszcza wielokrotnie, *list comprehension* jest mniej czytelne i może być problematyczne w poprawnym zapisie.)

Używając *list comprehension* zamiast pętli for należy stosować taki zapis:

`nowa_lista = [wyrażenie for element in isteniejąca_lista]`

Można również dodawać warunki:

`nowa_lista = [wyrażenie for element in isteniejąca_lista if warunek]`

Poniżej prosty przykład - załóżmy, że chcemy utworzyć listę liczb od 1 do 10, a następnie na jej podstawie utworzyć listę liczb parzystych i pomnożyć je przez 2. Standardowy zapis w pętli wyglądałby tak:

In [2]:
liczby = [] #deklaracja pustej listy - potrzebna do operacji append, która służy do dodawania elementu do listy
for i in range(1,11):
    liczby.append(i)
print('Liczby od 1 do 10:', liczby)
    
parzyste = []
for x in liczby:
    if x%2 == 0:
        parzyste.append(x*2)
print('Liczby parzyste x2:', parzyste)

Liczby od 1 do 10: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Liczby parzyste x2: [4, 8, 12, 16, 20]


Za pomocą *list comprehension* jest znacznie krócej:

In [3]:
liczby = list(range(1,11)) #range jest generatorem - przy wywołaniu zwraca kolejne elementy
print('Liczby od 1 do 10:', liczby)

parzyste = [x*2 for x in liczby if x%2 == 0]
print('Liczby parzyste x2:', parzyste)

Liczby od 1 do 10: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Liczby parzyste x2: [4, 8, 12, 16, 20]


Operacje można też prowadzić na dwóch listach równocześnie, np.

In [4]:
lista1 = ['a', 'b', 'c']
lista2 = ['1', '2', '3']

suma_list = [x+y for x, y in zip(lista1, lista2)]
print(suma_list)

['a1', 'b2', 'c3']


Sprawdź, co się stanie, jeśli jedna z list będzie dłuższa:

In [5]:
lista1 = ['a', 'b', 'c', 'd']
lista2 = ['1', '2', '3']

suma_list = [x+y for x, y in zip(lista1, lista2)]
print(suma_list)

['a1', 'b2', 'c3']


W analogiczny sposób zamiast list możemy tworzyć słowniki, używając *dictionary comprehension*:

In [6]:
keys = ['a', 'b', 'c', 'd']
values = [1, 2, 3, 4]
simple_dict = {key: value for key, value in zip(keys, values)} #wszystkie elementy dodane do słownika
print(simple_dict) 
simple_odd_dict = {key: value for key, value in zip(keys, values) if value%2 != 0} #nieparzyste elementy 
                                                                                #dodane do słownika
print(simple_odd_dict)

{'a': 1, 'b': 2, 'c': 3, 'd': 4}
{'a': 1, 'c': 3}


### 2. Zapisywanie do plików

Macierze i listy można zapisywać do pliku na wiele sposobów - można je zapisać jako plik możliwy do otworzenia w innych programach, np. .TXT,.CSV.

In [6]:
folder_path = Path('zapisywanie_do_plikow_przyklady')
os.makedirs(folder_path, exist_ok=True) #tworzymy folder, exist_ok - jeżeli taki istnieje, funkcja nie wyrzuca błędów

np.savetxt(folder_path / 'parzyste.txt', parzyste, fmt='%d') #fmt='%d' - format, w jakim mają być zapisane dane, 
                                            #d oznacza liczby całkowite (integer), można zamiast tego użyć fmt='%i'
pd.DataFrame(parzyste).to_csv(folder_path / 'parzyste.csv', index=False)

#zapis listy, każdy element w nowej linii - można w ten sposób zapisywać do pliku .TXT string, np. nazwy plików
with open(folder_path / 'lista1.txt', 'w') as text_file:
    for element in lista1:
        text_file.write(element+'\n')

Można też zapisać je do plików, których otwarcie będzie wymagało Pythona, np.:

In [7]:
np.save(folder_path / 'liczby_parzyste.npy', parzyste)
pd.DataFrame(parzyste).to_hdf(folder_path / 'parzyste.h5', key='df', mode='w') #pliki HDF/.H5 są często używane do przechowywania zbiorów danych
pickle.dump(parzyste, open(folder_path / 'parzyste_pickle','wb')) #wb oznacza write binary - zmienna parzyste zostanie zapisana do pliku binarnego

Żeby wczytać pliki, które przed chwilą zapisaliśmy, należy użyć funkcji:

In [8]:
parzyste_txt = np.loadtxt(folder_path / 'parzyste.txt')
parzyste_csv = pd.read_csv(folder_path / 'parzyste.csv') #uwaga, wczyta się jako DataFrame
parzyste_numpy = np.load(folder_path / 'liczby_parzyste.npy')
parzyste_pickle = pickle.load(open(folder_path / "parzyste_pickle", 'rb')) #rb oznacza read binary
parzyste_hdf = pd.read_hdf(folder_path / 'parzyste.h5', 'df')

Wyświetl powyższe 4 zmienne i zobacz, czym różni się różnią

Żeby wczytując z pliku .CSV lub HDF uzyskać zwykłą macierz, należy to zrobić w ten sposób:

In [9]:
parzyste_csv = pd.read_csv(folder_path / 'parzyste.csv').to_numpy().flatten()
parzyste_hdf = pd.read_hdf(folder_path / 'parzyste.h5', 'df').to_numpy().flatten()

Warto zapamiętac bibliotekę `pickle` - jest szczególnie użyteczna, bo pozwala na zapisanie do pliku dowolnej struktury danych, również klasyfikatorów, sieci neuronowych i innych modeli, które będziemy tworzyć w trakcie semestru. Używa się jej zawsze w ten sam sposób, bez względu na to, co chcemy zapisać.

Z tego samego powodu (możliwość zapisania dowolnej struktury obiektów) należy być ostrożnym przy otwieraniu plików w tym formacie z nieznanego źródła.

### 3. Biblioteka `os` i operacje na stringach

Ostatnia część powtórki dotyczy użycia biblioteki os oraz operacji na stringach, które mogą się przydać do pracy na plikach.

Dzięki bibliotece os jesteśmy w stanie np. utworzyć nowy folder (to już zrobiliśmy wyżej), wypisać listę wszystkich plików w danym folderze, strukturę folderów, utworzyć listę ścieżek dostępów do plików, które chcemy poddać analizie itp.

Poniżej kilka przykładów użycia:

In [10]:
#tworzenie listy ścieżek dostępu do plików, które znajdują się w folderze path = 'zapisywanie_do_plikow_przyklady/'
paths_to_files = []
for directory, subdirs, files in os.walk(folder_path):
    print(directory) #nazwa folderu głównego
    print(subdirs) #w tym przypadku pusta lista, ponieważ w folderze nie ma żadnych innych folderów
    print(files) #lista plików w folderze
    paths_to_files.extend([os.path.join(folder_path, file) for file in files])

zapisywanie_do_plikow_przyklady
[]
['parzyste.csv', 'lista1_renamed.txt', 'parzyste.h5', 'parzyste_pickle', 'parzyste_renamed.txt', 'lista1.txt', 'parzyste.txt', 'liczby_parzyste.npy']


In [11]:
#tworzenie listy plików o rozszerzeniu .TXT
paths_to_txt_files = []
for directory, subdirs, files in os.walk(folder_path):
    paths_to_txt_files.extend([os.path.join(folder_path, file) for file in files if file.endswith('.txt')])

#alternatywna metoda
paths_to_txt_files = glob(os.path.join(folder_path, '**/*.txt'), recursive=True)

In [12]:
#zmiana nazwy plików
for directory, subdirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.txt'):
            old_name = os.path.join(folder_path, file)
            file_name, extension = os.path.splitext(file)
            new_name = os.path.join(folder_path, f'{file_name}_renamed{extension}')
            os.rename(old_name, new_name)     

Użycie większości opisanych funkcji przećwiczymy w zadaniu Lab1_pierwszy_klasyfikator.